In [39]:
import pandas as pd 
import git
import numpy as np
from datetime import datetime as dt
from vehicle_stream_pipeline.utils import get_shortest_ride, get_hotspots, calculate_drives, calculate_graph

repo = git.Repo(".", search_parent_directories=True).git.rev_parse(
    "--show-toplevel")

In [2]:
drives = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")
drives = drives[(drives["state"] == "completed")]
edges = pd.read_excel(f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name= 'Liste 2022')

/var/folders/gb/hn_j3jkn5_5fm5ylgtcd9w_00000gn/T/ipykernel_47624/3440087016.py:1: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  drives = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")


In [3]:
startdate = drives.scheduled_to.min()
startdate = dt.strptime(startdate, '%Y-%m-%d %H:%M:%S')
startdate = startdate.strftime('%m/%d/%Y')
start_date = dt.strptime(startdate, "%m/%d/%Y")
enddate = drives.scheduled_to.max()
enddate = dt.strptime(enddate, '%Y-%m-%d %H:%M:%S')
enddate = enddate.strftime('%m/%d/%Y')
end_date = dt.strptime(enddate, "%m/%d/%Y")

In [4]:
aggregated_drives = calculate_drives(drives, start_date, end_date)

In [5]:
graph = calculate_graph(aggregated_drives)

In [19]:
hotspots = get_hotspots(edges, aggregated_drives)

In [32]:
hotspots = [spot[0] for spot in hotspots]

In [40]:
def add_drone_flights(df_edges, drives, drone_spots = [1008], radius = 500):
    drone_flights = df_edges.iloc[:, :6]
    drone_flights.drop(
        ['Start Name', 'Ende Name', 'Route [m]'], axis=1, inplace=True)
    drone_flights.rename(columns={'Luftlinie [m]': 'Luftlinie'}, inplace=True)
    drone_flights = drone_flights[drone_flights.Luftlinie <= radius]

    drone_flights['avg_time_to_destination'] = (
        drone_flights.Luftlinie * 0.12) / 60 / 60 / 24

    drone_flights = drone_flights[(drone_flights["start_id"].isin(drone_spots)) | (drone_flights["end_id"].isin(drone_spots))]
    
    drives_w_flights = pd.merge(drives, drone_flights, left_on=[
                                "pickup_address", "dropoff_address"], right_on=["start_id", "end_id"], how="left")

    drives_w_flights['avg_time_to_destination'] = np.where((drives_w_flights['avg_time_to_destination_x'] > drives_w_flights['avg_time_to_destination_y']),
                                                           drives_w_flights.avg_time_to_destination_y,
                                                           drives_w_flights.avg_time_to_destination_x)
    
    return drives_w_flights

In [41]:
drives_w_flights = add_drone_flights(edges, aggregated_drives, drone_spots = hotspots)

AttributeError: module 'pandas' has no attribute 'where'

In [36]:
drives_w_flights

,start_id,end_id,Luftlinie,avg_time_to_destination
8,1001,1010,314,0.000436
203,1002,1005,262,0.000364
207,1002,1009,430,0.000597
208,1002,1010,257,0.000357
403,1003,1005,200,0.000278
...,...,...,...,...
31757,12007,12006,395,0.000549
31758,12007,12008,239,0.000332
31760,12007,12010,459,0.000637
31958,12008,12007,239,0.000332
